# 🚀 Medical MLOps: Hands-On Practice

## Table of Contents
1. [Container Setup with Docker](#practice-1-container-setup-with-docker)
2. [Model Registry with MLflow](#practice-2-model-registry-with-mlflow)
3. [Data Versioning with DVC](#practice-3-data-versioning-with-dvc)
4. [CI/CD Pipeline Setup](#practice-4-cicd-pipeline-setup)
5. [Model Performance Monitoring](#practice-5-model-performance-monitoring)
6. [Data Drift Detection](#practice-6-data-drift-detection)
7. [Complete MLOps Pipeline](#practice-7-complete-mlops-pipeline)

## Installing and Importing Essential Libraries

In [ ]:
# Install required packages (run once)
# !pip install mlflow scikit-learn numpy pandas matplotlib seaborn scipy

# Import essential libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy import stats
import mlflow
import mlflow.sklearn
import warnings
warnings.filterwarnings('ignore')

# Visualization settings
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12
sns.set_style('whitegrid')

print("✅ All libraries loaded successfully!")

---
## Practice 1: Container Setup with Docker

### 🎯 Learning Objectives
- Understand containerization concepts
- Create a simple Dockerfile for ML models
- Learn reproducible environment setup

### 📖 Key Concepts
**Docker**: Platform for packaging applications with all dependencies into containers  
**Benefits**: Reproducibility, Isolation, Portability

In [ ]:
# 1.1 Create a simple Dockerfile template
def create_dockerfile_template():
    """Generate a Dockerfile template for ML model deployment"""
    
    dockerfile_content = """
# Dockerfile for Medical ML Model
FROM python:3.9-slim

# Set working directory
WORKDIR /app

# Copy requirements
COPY requirements.txt .

# Install dependencies
RUN pip install --no-cache-dir -r requirements.txt

# Copy application code
COPY . .

# Expose port
EXPOSE 8000

# Run application
CMD ["python", "app.py"]
"""
    
    requirements_content = """
numpy==1.24.3
pandas==2.0.3
scikit-learn==1.3.0
mlflow==2.8.0
fastapi==0.104.1
uvicorn==0.24.0
"""
    
    print("📦 Docker Configuration")
    print("=" * 60)
    print("\n[Dockerfile]")
    print(dockerfile_content)
    print("\n[requirements.txt]")
    print(requirements_content)
    
    print("\n💡 Docker Commands:")
    print("  Build: docker build -t medical-ml-model .")
    print("  Run:   docker run -p 8000:8000 medical-ml-model")
    print("  Push:  docker push registry/medical-ml-model:v1.0")
    
    return dockerfile_content, requirements_content

dockerfile, requirements = create_dockerfile_template()

---
## Practice 2: Model Registry with MLflow

### 🎯 Learning Objectives
- Track model experiments with MLflow
- Version and register models
- Compare model performance across versions

In [ ]:
# 2.1 Generate synthetic medical data
def generate_medical_data(n_samples=1000):
    """Create synthetic medical diagnostic data"""
    
    np.random.seed(42)
    
    # Generate features (patient measurements)
    X, y = make_classification(
        n_samples=n_samples,
        n_features=10,
        n_informative=8,
        n_redundant=2,
        n_classes=2,
        weights=[0.7, 0.3],
        random_state=42
    )
    
    # Create DataFrame
    feature_names = [f'Feature_{i+1}' for i in range(10)]
    df = pd.DataFrame(X, columns=feature_names)
    df['Diagnosis'] = y  # 0: Negative, 1: Positive
    
    print("🏥 Synthetic Medical Dataset Generated")
    print("=" * 60)
    print(f"Total samples: {len(df)}")
    print(f"Features: {len(feature_names)}")
    print(f"\nClass distribution:")
    print(df['Diagnosis'].value_counts())
    print(f"\nFirst 5 rows:")
    print(df.head())
    
    return df

medical_data = generate_medical_data()

In [ ]:
# 2.2 Train and log model with MLflow
def train_and_log_model(data, model_name="Medical_Diagnosis_Model"):
    """Train a model and track with MLflow"""
    
    # Prepare data
    X = data.drop('Diagnosis', axis=1)
    y = data['Diagnosis']
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    # Set MLflow experiment
    mlflow.set_experiment("Medical_MLOps_Demo")
    
    print("🔬 Training Model with MLflow Tracking")
    print("=" * 60)
    
    with mlflow.start_run(run_name="RandomForest_v1"):
        # Train model
        model = RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            random_state=42
        )
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        # Log parameters
        mlflow.log_param("n_estimators", 100)
        mlflow.log_param("max_depth", 10)
        mlflow.log_param("model_type", "RandomForest")
        
        # Log metrics
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)
        
        # Log model
        mlflow.sklearn.log_model(model, "model")
        
        print("\n📊 Model Performance:")
        print(f"  Accuracy:  {accuracy:.4f}")
        print(f"  Precision: {precision:.4f}")
        print(f"  Recall:    {recall:.4f}")
        print(f"  F1 Score:  {f1:.4f}")
        
        print("\n✅ Model logged to MLflow successfully!")
        print("💡 View results: mlflow ui")
    
    return model, X_test, y_test, y_pred

model, X_test, y_test, y_pred = train_and_log_model(medical_data)

---
## Practice 3: Data Versioning with DVC

### 🎯 Learning Objectives
- Understand the importance of data versioning
- Learn DVC commands and workflow
- Track dataset changes over time

In [ ]:
# 3.1 DVC workflow demonstration
def demonstrate_dvc_workflow():
    """Show DVC commands for data versioning"""
    
    dvc_workflow = """
📦 Data Version Control (DVC) Workflow
==========================================

1. Initialize DVC:
   $ dvc init
   $ git add .dvc .dvcignore
   $ git commit -m "Initialize DVC"

2. Add data to DVC tracking:
   $ dvc add data/medical_images.zip
   $ git add data/medical_images.zip.dvc data/.gitignore
   $ git commit -m "Add medical images dataset v1.0"

3. Configure remote storage:
   $ dvc remote add -d storage s3://mybucket/dvc-storage
   $ git add .dvc/config
   $ git commit -m "Configure DVC remote storage"

4. Push data to remote:
   $ dvc push

5. Pull data from remote:
   $ dvc pull

6. Checkout specific version:
   $ git checkout v1.0 data/medical_images.zip.dvc
   $ dvc checkout

💡 Benefits:
   - Version large datasets efficiently
   - Reproducible experiments
   - Collaborate on data without Git bloat
   - Track data lineage
"""
    
    print(dvc_workflow)
    
    # Save dataset with version info
    dataset_info = {
        'version': '1.0.0',
        'samples': len(medical_data),
        'features': len(medical_data.columns) - 1,
        'date': pd.Timestamp.now().strftime('%Y-%m-%d'),
        'description': 'Initial medical diagnostic dataset'
    }
    
    print("\n📋 Current Dataset Version:")
    for key, value in dataset_info.items():
        print(f"  {key}: {value}")
    
    return dataset_info

dataset_version = demonstrate_dvc_workflow()

---
## Practice 4: CI/CD Pipeline Setup

### 🎯 Learning Objectives
- Understand CI/CD concepts for ML
- Create automated testing pipeline
- Implement deployment strategies

In [ ]:
# 4.1 Create GitHub Actions workflow
def create_cicd_pipeline():
    """Generate CI/CD pipeline configuration"""
    
    github_actions = """
# .github/workflows/ml-pipeline.yml
name: ML Model CI/CD

on:
  push:
    branches: [ main ]
  pull_request:
    branches: [ main ]

jobs:
  test:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      
      - name: Set up Python
        uses: actions/setup-python@v4
        with:
          python-version: '3.9'
      
      - name: Install dependencies
        run: |
          pip install -r requirements.txt
          pip install pytest
      
      - name: Run unit tests
        run: pytest tests/
      
      - name: Check model performance
        run: python scripts/validate_model.py
  
  deploy:
    needs: test
    runs-on: ubuntu-latest
    if: github.ref == 'refs/heads/main'
    steps:
      - name: Deploy to staging
        run: |
          echo "Deploying to staging environment"
          # kubectl apply -f k8s/staging/
      
      - name: Run integration tests
        run: python scripts/integration_test.py
      
      - name: Deploy to production
        run: |
          echo "Deploying to production"
          # kubectl apply -f k8s/production/
"""
    
    print("🔄 CI/CD Pipeline Configuration")
    print("=" * 60)
    print(github_actions)
    
    # Pipeline stages visualization
    stages = [
        "1️⃣ Code Commit → GitHub",
        "2️⃣ Build & Test → Run pytest",
        "3️⃣ Model Validation → Check metrics",
        "4️⃣ Deploy to Staging → Test environment",
        "5️⃣ Integration Tests → End-to-end validation",
        "6️⃣ Deploy to Production → Live system"
    ]
    
    print("\n📊 Pipeline Stages:")
    for stage in stages:
        print(f"  {stage}")
    
    return github_actions

cicd_config = create_cicd_pipeline()

---
## Practice 5: Model Performance Monitoring

### 🎯 Learning Objectives
- Track model metrics over time
- Detect performance degradation
- Set up alerting thresholds

In [ ]:
# 5.1 Simulate model performance over time
def simulate_model_monitoring(n_days=30):
    """Simulate model performance metrics over time"""
    
    np.random.seed(42)
    
    # Simulate metrics with slight degradation
    days = np.arange(1, n_days + 1)
    accuracy = 0.95 - 0.001 * days + np.random.normal(0, 0.01, n_days)
    latency = 100 + 2 * days + np.random.normal(0, 5, n_days)  # milliseconds
    error_rate = 0.05 + 0.001 * days + np.random.normal(0, 0.005, n_days)
    
    # Create monitoring DataFrame
    monitoring_data = pd.DataFrame({
        'Day': days,
        'Accuracy': accuracy,
        'Latency_ms': latency,
        'Error_Rate': error_rate
    })
    
    # Plot metrics
    fig, axes = plt.subplots(3, 1, figsize=(12, 10))
    
    # Accuracy plot
    axes[0].plot(monitoring_data['Day'], monitoring_data['Accuracy'], 
                 marker='o', color='blue', alpha=0.7)
    axes[0].axhline(y=0.92, color='red', linestyle='--', label='Alert Threshold')
    axes[0].set_xlabel('Day')
    axes[0].set_ylabel('Accuracy')
    axes[0].set_title('Model Accuracy Over Time')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Latency plot
    axes[1].plot(monitoring_data['Day'], monitoring_data['Latency_ms'], 
                 marker='s', color='green', alpha=0.7)
    axes[1].axhline(y=150, color='red', linestyle='--', label='SLA Threshold')
    axes[1].set_xlabel('Day')
    axes[1].set_ylabel('Latency (ms)')
    axes[1].set_title('Inference Latency Over Time')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    # Error rate plot
    axes[2].plot(monitoring_data['Day'], monitoring_data['Error_Rate'], 
                 marker='^', color='orange', alpha=0.7)
    axes[2].axhline(y=0.10, color='red', linestyle='--', label='Alert Threshold')
    axes[2].set_xlabel('Day')
    axes[2].set_ylabel('Error Rate')
    axes[2].set_title('System Error Rate Over Time')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Check for alerts
    print("\n⚠️ Alert Configuration:")
    print("=" * 60)
    
    alerts = []
    if monitoring_data['Accuracy'].iloc[-1] < 0.92:
        alerts.append("🚨 Accuracy dropped below 92%")
    if monitoring_data['Latency_ms'].iloc[-1] > 150:
        alerts.append("🚨 Latency exceeded 150ms SLA")
    if monitoring_data['Error_Rate'].iloc[-1] > 0.10:
        alerts.append("🚨 Error rate exceeded 10%")
    
    if alerts:
        for alert in alerts:
            print(f"  {alert}")
    else:
        print("  ✅ All metrics within normal range")
    
    return monitoring_data

monitoring_data = simulate_model_monitoring()

---
## Practice 6: Data Drift Detection

### 🎯 Learning Objectives
- Detect distribution changes in input data
- Apply statistical tests (KS test, Chi-square)
- Trigger model retraining when needed

In [ ]:
# 6.1 Detect data drift using statistical tests
def detect_data_drift(original_data, new_data, threshold=0.05):
    """Detect drift using Kolmogorov-Smirnov test"""
    
    print("🔍 Data Drift Detection")
    print("=" * 60)
    
    drift_detected = False
    feature_drifts = {}
    
    for column in original_data.columns:
        if column == 'Diagnosis':
            continue
        
        # Perform KS test
        statistic, p_value = stats.ks_2samp(
            original_data[column],
            new_data[column]
        )
        
        feature_drifts[column] = {
            'statistic': statistic,
            'p_value': p_value,
            'drift': p_value < threshold
        }
        
        if p_value < threshold:
            drift_detected = True
    
    # Print results
    print("\n📊 Feature Drift Analysis:")
    print(f"{'Feature':<15} {'KS Statistic':<15} {'P-Value':<12} {'Drift?'}")
    print("-" * 60)
    
    for feature, metrics in feature_drifts.items():
        drift_status = "⚠️ YES" if metrics['drift'] else "✅ NO"
        print(f"{feature:<15} {metrics['statistic']:<15.4f} "
              f"{metrics['p_value']:<12.4f} {drift_status}")
    
    print("\n" + "=" * 60)
    if drift_detected:
        print("🚨 Data drift detected! Consider retraining the model.")
    else:
        print("✅ No significant drift detected.")
    
    return feature_drifts, drift_detected

# Generate new data with slight drift
def generate_drifted_data(n_samples=500):
    """Generate data with distribution shift"""
    np.random.seed(100)
    
    X, y = make_classification(
        n_samples=n_samples,
        n_features=10,
        n_informative=8,
        n_redundant=2,
        n_classes=2,
        weights=[0.6, 0.4],  # Changed distribution
        random_state=100
    )
    
    # Add slight shift to features
    X = X + np.random.normal(0.3, 0.1, X.shape)
    
    feature_names = [f'Feature_{i+1}' for i in range(10)]
    df = pd.DataFrame(X, columns=feature_names)
    df['Diagnosis'] = y
    
    return df

new_data = generate_drifted_data()
drift_results, has_drift = detect_data_drift(medical_data, new_data)

---
## Practice 7: Complete MLOps Pipeline

### 🎯 Learning Objectives
- Integrate all MLOps components
- Create end-to-end workflow
- Understand production deployment cycle

In [ ]:
# 7.1 Complete MLOps pipeline implementation
def complete_mlops_pipeline():
    """Demonstrate complete MLOps workflow"""
    
    pipeline_steps = """
🚀 Complete MLOps Pipeline
==========================================

Step 1: Data Versioning
  ✅ Track datasets with DVC
  ✅ Version control with Git
  ✅ Store in cloud (S3/Azure/GCS)

Step 2: Model Training
  ✅ Train model with versioned data
  ✅ Log experiments to MLflow
  ✅ Track hyperparameters and metrics

Step 3: Model Registry
  ✅ Register model in MLflow
  ✅ Tag with metadata (version, author, date)
  ✅ Stage: Development → Staging → Production

Step 4: Containerization
  ✅ Create Dockerfile
  ✅ Build Docker image
  ✅ Push to container registry

Step 5: CI/CD Pipeline
  ✅ Automated testing (unit, integration)
  ✅ Model validation checks
  ✅ Gradual rollout (canary/blue-green)

Step 6: Deployment
  ✅ Deploy to Kubernetes cluster
  ✅ Configure auto-scaling
  ✅ Set up load balancing

Step 7: Monitoring
  ✅ Track performance metrics
  ✅ Detect data drift
  ✅ Alert on anomalies
  ✅ Collect audit logs

Step 8: Maintenance
  ✅ Regular model retraining
  ✅ A/B testing new versions
  ✅ Rollback if needed
  ✅ Continuous improvement
"""
    
    print(pipeline_steps)
    
    # Create visualization of pipeline
    fig, ax = plt.subplots(figsize=(14, 8))
    
    stages = [
        'Data\nVersioning',
        'Model\nTraining',
        'Model\nRegistry',
        'Container\nization',
        'CI/CD\nPipeline',
        'Deploy\nment',
        'Monitor\ning',
        'Main\ntenance'
    ]
    
    x_positions = np.arange(len(stages))
    colors = plt.cm.viridis(np.linspace(0.2, 0.9, len(stages)))
    
    # Plot stages
    for i, (stage, color) in enumerate(zip(stages, colors)):
        ax.bar(i, 1, color=color, alpha=0.7, edgecolor='black', linewidth=2)
        ax.text(i, 0.5, stage, ha='center', va='center', 
                fontsize=11, fontweight='bold', color='white')
        
        # Add arrows
        if i < len(stages) - 1:
            ax.annotate('', xy=(i + 0.5, 0.5), xytext=(i + 0.5, 0.5),
                       arrowprops=dict(arrowstyle='->', lw=3, color='gray'))
    
    ax.set_xlim(-0.5, len(stages) - 0.5)
    ax.set_ylim(0, 1.2)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title('Complete MLOps Pipeline Flow', fontsize=16, fontweight='bold', pad=20)
    ax.spines['top'].set_visible(False)
    ax.spines('right').set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    
    plt.tight_layout()
    plt.show()
    
    print("\n💡 Key Takeaways:")
    print("  • MLOps is more than just deploying a model")
    print("  • Automation and monitoring are critical")
    print("  • Version everything: data, code, models")
    print("  • Continuous improvement through feedback loops")
    print("  • Healthcare requires extra compliance and governance")

complete_mlops_pipeline()

---
## 🎯 Practice Complete!

### Summary of What We Learned:

1. **Containerization**: Packaging models with Docker for reproducibility
2. **Model Registry**: Tracking experiments and versions with MLflow
3. **Data Versioning**: Managing datasets with DVC
4. **CI/CD**: Automating testing and deployment workflows
5. **Monitoring**: Tracking performance metrics in production
6. **Drift Detection**: Identifying when models need retraining
7. **Complete Pipeline**: Integrating all components for production ML

### Key Insights:
- **MLOps ≠ Just Deployment**: It's a complete lifecycle management system
- **Automation is Key**: Reduce manual errors and speed up iterations
- **Monitoring is Critical**: Catch issues before they impact users
- **Version Everything**: Data, code, models, and infrastructure

### Next Steps:
1. Set up a complete MLOps environment locally
2. Practice with real medical datasets (MIMIC, PhysioNet)
3. Learn Kubernetes for orchestration
4. Explore advanced monitoring with Prometheus + Grafana
5. Study healthcare compliance (HIPAA, FDA, GDPR)

### Additional Resources:
- MLflow Documentation: https://mlflow.org/
- DVC Documentation: https://dvc.org/
- Kubernetes Documentation: https://kubernetes.io/
- Medical MLOps Papers: Search on Papers with Code

---
## 📝 Exercise: Build Your Own Pipeline

### Challenge
Using the concepts learned, create a complete MLOps pipeline for a medical image classification task:

1. Load a medical image dataset
2. Train a CNN model
3. Log experiments to MLflow
4. Create a Docker container
5. Set up monitoring metrics
6. Implement drift detection

**Bonus**: Deploy to a cloud platform (AWS/GCP/Azure) and set up CI/CD!